Loading dataset using Tensorflow Dataset API

In [5]:
import tensorflow as tf
import tensorflow_datasets as tfds

In [10]:
(ds_train, ds_test), ds_info = tfds.load('imdb_reviews', 
                                        split=(tfds.Split.TRAIN, tfds.Split.TEST),
                                        as_supervised=True,
                                        with_info=True)

Now check some examples from the dataset.

In [11]:
for review, label in tfds.as_numpy(ds_train.take(5)):
    print(review.decode()[0:50],'\t', label)

This was an absolutely terrible movie. Don't be lu 	 0
I have been known to fall asleep during films, but 	 0
Mann photographs the Alberta Rocky Mountains in a  	 0
This is the kind of film for a snowy Sunday aftern 	 1
As others have mentioned, all the women that go nu 	 1


Now we need to apply BERT tokenizer to use pre-trained tokenizer.

In [12]:
from transformers import BertTokenizer

In [13]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case = True)

Now we need to prepare the data for BERT model.

Input IDs – The input ids are often the only required parameters to be passed to the model as input. Token indices, numerical representations of tokens building the sequences that will be used as input by the model.

Attention mask – Attention Mask is used to avoid performing attention on padding token indices. Mask value can be either 0 or 1, 1 for tokens that are NOT MASKED, 0 for MASKED tokens.

Token type ids – It is used in use cases like sequence classification or question answering. As these require two different sequences to be encoded in the same input IDs. Special tokens, such as the classifier[CLS] and separator[SEP] tokens are used to separate the sequences.

<center><img src="img1.png"/></center>
<center>Source: BERT Input/Output(https://pysnacks.com/machine-learning/bert-text-classification-with-fine-tuning/)</center>

The encode_plus function of the tokenizer class will tokenize the raw input.

In [6]:
max_length = 512
def convert_to_feature(review):
    return tokenizer.encode_plus(review,
                    add_special_tokens=True, # add [CLS], [SEP]
                    max_length=max_length, # max length of the text that can go to BERT
                    pad_to_max_length=True, # add [PAD] tokens 
                    return_attention_mask=True) # add attention mask to not focus on pad tokens

Following helper functions are transforming raw data to an appropriate format to feed into the BERT model.

In [3]:
def map_to_dict(input_ids, attention_masks, token_type_ids, label):
    return {
        "input_ids":input_ids,
        "attention_maks":attention_masks,
        "token_typ_ids": token_type_ids
    }, label

In [17]:
def encode_reviews(ds, limit=-1):
    input_ids_list=[]
    token_type_ids_list=[]
    attention_mask_list=[]
    label_list=[]
    if limit>0:
        ds=ds.take(limit)
    
    for review, label in tfds.as_numpy(ds):
        bert_input = convert_to_feature(review.decode())
        input_ids_list.append(bert_input['input_ids'])
        token_type_ids_list.append(bert_input['token_type_ids'])
        attention_mask_list.append(bert_input['attention_mask'])
        label_list.append(label)
    
    return tf.data.Dataset.from_tensor_slices((input_ids_list, attention_mask_list, token_type_ids_list,label_list)).map(map_to_dict)


Now we need to create our train and test datasets.

In [18]:
batch_size=6
ds_train_encoded = encode_reviews(ds_train).shuffle(10000).batch(batch_size)
ds_test_encoded = encode_reviews(ds_test).batch(batch_size)

Now we will initialize BERT model for sentiment analysis.

In [19]:
from transformers import TFBertForSequenceClassification
learning_rate = 2e-5
epoch = 2
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased')

optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate, epsilon=1e-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric]) 

Downloading:   0%|          | 0.00/536M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Now we will train the model.

In [ ]:
bert_history = model.fit(ds_train_encoded, epochs=epoch, validation_data=ds_test_encoded)